In [ ]:
#Carregar o arquivo input_data.py
import input_data

In [ ]:
mnist = input_data.read_data_sets('./mnist/', one_hot = True)
X = mnist.train.images

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(X[1].reshape(28,28))

In [ ]:
# 784 -> 128 -> 64 -> 128 -> 784

In [ ]:
# encoder
neuronios_entrada = 784
neuronios_oculta1 = 128

# dado/imagem codificada
neuronios_oculta2 = 64

# decoder
neuronios_oculta3 = neuronios_oculta1
neuronios_saida = neuronios_entrada

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.reset_default_graph()

In [ ]:
xph = tf.placeholder(tf.float32, [None, neuronios_entrada])

In [ ]:
# Xavier: sigmoid
# He: relu
inicializador = tf.variance_scaling_initializer()

In [ ]:
# 784 -> 128 -> 64 -> 128 -> 784
W = {'encoder_oculta1': tf.Variable(inicializador([neuronios_entrada, neuronios_oculta1])),
     'encoder_oculta2': tf.Variable(inicializador([neuronios_oculta1, neuronios_oculta2])),
     'decoder_oculta3': tf.Variable(inicializador([neuronios_oculta2, neuronios_oculta3])),
     'decoder_saida': tf.Variable(inicializador([neuronios_oculta3, neuronios_saida]))
}

In [ ]:
b = {'encoder_oculta1': tf.Variable(inicializador([neuronios_oculta1])),
     'encoder_oculta2': tf.Variable(inicializador([neuronios_oculta2])),
     'decoder_oculta3': tf.Variable(inicializador([neuronios_oculta3])),
     'decoder_saida': tf.Variable(inicializador([neuronios_saida]))
}

In [ ]:
camada_oculta1 = tf.nn.relu(tf.add(tf.matmul(xph, W['encoder_oculta1']), b['encoder_oculta1']))
camada_oculta2 = tf.nn.relu(tf.add(tf.matmul(camada_oculta1, W['encoder_oculta2']), b['encoder_oculta2']))
camada_oculta3 = tf.nn.relu(tf.add(tf.matmul(camada_oculta2, W['decoder_oculta3']), b['decoder_oculta3']))
camada_saida = tf.nn.relu(tf.add(tf.matmul(camada_oculta3, W['decoder_saida']), b['decoder_saida']))

In [ ]:
erro = tf.losses.mean_squared_error(xph, camada_saida)
otimizador = tf.train.AdamOptimizer(learning_rate = 0.001)
treinamento = otimizador.minimize(erro)
batch_size = 128

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoca in range(50):
        numero_batches = mnist.train.num_examples // batch_size
        for i in range(numero_batches):
            X_batch, _ = mnist.train.next_batch(batch_size)
            custo, _ = sess.run([erro, treinamento], feed_dict = {xph: X_batch})
        print('época: ' + str(epoca + 1) + ' erro: ' + str(custo))
    
    imagens_codificadas = sess.run(camada_oculta2, feed_dict = {xph: X})
    imagens_decodificadas = sess.run(camada_saida, feed_dict = {xph: X})

In [ ]:
imagens_codificadas.shape

In [ ]:
imagens_codificadas[0]

In [ ]:
imagens_decodificadas.shape

In [ ]:
imagens_decodificadas[0]

In [ ]:
import numpy as np
numero_imagens = 5
imagens_teste = np.random.randint(X.shape[0], size = numero_imagens)
imagens_teste

In [ ]:
plt.figure(figsize = (18, 18))
for i, indice_imagem in enumerate(imagens_teste):
    #print(i)
    #print(indice_imagem)
    eixo = plt.subplot(10, 5, i + 1)
    plt.imshow(X[indice_imagem].reshape(28, 28))
    plt.xticks(())
    plt.yticks(())
    
    eixo = plt.subplot(10, 5, i + 1 + numero_imagens)
    plt.imshow(imagens_codificadas[indice_imagem].reshape(8, 8))
    plt.xticks(())
    plt.yticks(())
    
    eixo = plt.subplot(10, 5, i + 1 + numero_imagens * 2)
    plt.imshow(imagens_decodificadas[indice_imagem].reshape(28, 28))
    plt.xticks(())
    plt.yticks(())
    
    